In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import pearsonr, ttest_ind
import sys
sys.path.append("..")
from dataset import get_loaders

In [2]:
data = get_loaders(batch_size=32, meta=False,
                    use_images=True, image_dir='../images/',
                    n_episodes=None)
train_data, train_loader, test_data, test_loader, analyze_data, analyze_loader = data


In [3]:
idx2loc = {idx:loc for loc, idx in test_data.loc2idx.items()}

In [12]:
with open('../../results/results_rnn.P', 'rb') as f:
    results = pickle.load(f)
print(results.keys())

dict_keys(['Episodic', 'Cortical'])


In [13]:
analyze_correct = results['Cortical']['analyze_correct']

In [16]:
hidd_t_idx = 1 # at what time step, t = 1 means at the time of face1 
                              # and t = 2 means at the time of face2
# create groups based on the row or columns
# e.g, for context0 (xaxis), first column is group 1, sec col is group 2, and so on.
# 4 groups for each axis/context; total 8 groups

ctx0_g0=[]
ctx0_g1=[]
ctx0_g2=[]
ctx0_g3=[]

ctx1_g0=[]
ctx1_g1=[]
ctx1_g2=[]
ctx1_g3=[]

for i, batch in enumerate(analyze_loader):
    f1, f2, ax, y, idx1, idx2 = batch
    acc = analyze_correct[i][hidd_t_idx]
    ax = ax.cpu().numpy().squeeze()
    idx1 = idx1[0]
    idx2 = idx2[0]
    loc1 = idx2loc[idx1]
    loc2 = idx2loc[idx2]
    if ax==0:
        if loc1[ax]==0: ctx0_g0.append(acc) # (len(all_perms)/2) / 4 = [48]
        elif loc1[ax]==1: ctx0_g1.append(acc)
        elif loc1[ax]==2: ctx0_g2.append(acc)
        elif loc1[ax]==3: ctx0_g3.append(acc)
    elif ax==1:
        if loc1[ax]==0: ctx1_g0.append(acc)
        elif loc1[ax]==1: ctx1_g1.append(acc)
        elif loc1[ax]==2: ctx1_g2.append(acc)
        elif loc1[ax]==3: ctx1_g3.append(acc)
ctx0_accs = [np.mean(ctx0_g0), np.mean(ctx0_g1), 
            np.mean(ctx0_g2), np.mean(ctx0_g3) ]
ctx1_accs = [np.mean(ctx1_g0), np.mean(ctx1_g1), 
            np.mean(ctx1_g2), np.mean(ctx1_g3) ]
            
print('Accuracy at t=%s (face1) contex 0:' %(hidd_t_idx), ctx0_accs)
print('Accuracy at t=%s (face1) contex 1:' %(hidd_t_idx), ctx1_accs)

Accuracy at t=1 (face1) contex 0: [0.0, 0.3333333333333333, 0.6666666666666666, 0.75]
Accuracy at t=1 (face1) contex 1: [0.0, 0.5, 0.4166666666666667, 0.25]
